In [1]:
# import libraries (you may add additional imports but you may not have to)
import numpy as np
import pandas as pd
from scipy.sparse import csr_matrix
from sklearn.neighbors import NearestNeighbors
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler


In [2]:
# get data files
!wget https://cdn.freecodecamp.org/project-data/books/book-crossings.zip

!unzip book-crossings.zip

books_filename = 'BX-Books.csv'
ratings_filename = 'BX-Book-Ratings.csv'

--2025-03-13 21:01:40--  https://cdn.freecodecamp.org/project-data/books/book-crossings.zip
Resolving cdn.freecodecamp.org (cdn.freecodecamp.org)... 104.26.2.33, 104.26.3.33, 172.67.70.149, ...
Connecting to cdn.freecodecamp.org (cdn.freecodecamp.org)|104.26.2.33|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 26085508 (25M) [application/zip]
Saving to: ‘book-crossings.zip.4’

book-crossings.zip. 100%[===================>]  24.88M   160MB/s    in 0.2s    

2025-03-13 21:01:41 (160 MB/s) - ‘book-crossings.zip.4’ saved [26085508/26085508]

Archive:  book-crossings.zip
replace BX-Book-Ratings.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: N


In [3]:
# import csv data into dataframes
df_books = pd.read_csv(
    books_filename,
    encoding = "ISO-8859-1",
    sep=";",
    header=0,
    names=['isbn', 'title', 'author'],
    usecols=['isbn', 'title', 'author'],
    dtype={'isbn': 'str', 'title': 'str', 'author': 'str'})

df_ratings = pd.read_csv(
    ratings_filename,
    encoding = "ISO-8859-1",
    sep=";",
    header=0,
    names=['user', 'isbn', 'rating'],
    usecols=['user', 'isbn', 'rating'],
    dtype={'user': 'int32', 'isbn': 'str', 'rating': 'float32'})

In [8]:
#1 remove instances from the dataset users with less than 200 ratings and books with less than 100 ratings.
#user_counts = df_ratings['user'].value_counts()
#book_counts = df_ratings['isbn'].value_counts()
#print(user_counts)

#filtered_users = user_counts[user_counts >= 200].index
#df_ratings = df_ratings[df_ratings['user'].isin(filtered_users)]

#filtered_books = book_counts[book_counts >= 100].index
#df_ratings = df_ratings[df_ratings['isbn'].isin(filtered_books)]

#df_books = df_books[df_books['isbn'].isin(df_ratings['isbn'])]

df = df_ratings

user_counts = df['user'].value_counts()
book_counts = df['isbn'].value_counts()

df = df[~df['user'].isin(user_counts[user_counts < 200].index)]
df = df[~df['isbn'].isin(book_counts[book_counts < 100].index)]

df = pd.merge(right=df, left = df_books, on="isbn")

df.drop_duplicates(subset=["title", "user"], inplace=True)



,isbn,title,author,user,rating,title_x,author_x,title_y,author_y


In [9]:
#book_pivot = df_ratings.pivot_table(index='isbn', columns='user', values='rating', fill_value=0)
#book_pivot = df_ratings.pivot(index='isbn', columns='user', values='rating').fillna(0)
book_pivot = df.pivot(index='title', columns='user', values='rating')

book_pivot.fillna(0,inplace=True)
#book_pivot = df_ratings.pivot(index='isbn', columns='user', values='rating').fillna(0) #Check this
isbn_to_title = df_books.set_index('isbn')['title'].to_dict()
#scaler = MinMaxScaler()
#book_pivot_scaled = pd.DataFrame(scaler.fit_transform(book_pivot), index=book_pivot.index, columns=book_pivot.columns)

nn_model = NearestNeighbors(n_neighbors=5, algorithm='auto', metric='euclidean')
nn_model.fit(book_pivot)


ValueError: at least one array or dtype is required

In [ ]:
# function to return recommended books - this will be tested
def get_recommends(book = ""):
  book_isbn = df_books[df_books['title'] == book]['isbn'].values[0]
  print("Book Pivot Shape:", book_pivot.shape)
  print("Book ISBN Exists in Pivot:", book_isbn in book_pivot.index)
  print(type(book_pivot.loc[book_isbn].values.reshape(1, -1)))
  distances, indices = nn_model.kneighbors(book_pivot.loc[book_isbn].values.reshape(1, -1))
  print("Distances:", distances)

  recommended_isbns = book_pivot.index[indices[0]].tolist()

  recommended_books = []
  recommended_books.append(book)

  recommendations = []

  #for idx in range(1,len(recommended_isbns)):
    #recommendations.append([isbn_to_title[recommended_isbns[idx]], distances[0][idx]])
  recommendations = [
        [isbn_to_title[book_pivot.index[i]], distances[0][idx]]
        for idx, i in enumerate(indices[0][1:], start=1)  # Skip the first index (input book itself)
  ]
  recommended_books.append(recommendations)
  return recommended_books

In [ ]:
books = get_recommends("Where the Heart Is (Oprah's Book Club (Paperback))")
print(books)

def test_book_recommendation():
  test_pass = True
  recommends = get_recommends("Where the Heart Is (Oprah's Book Club (Paperback))")
  if recommends[0] != "Where the Heart Is (Oprah's Book Club (Paperback))":
    test_pass = False
  recommended_books = ["I'll Be Seeing You", 'The Weight of Water', 'The Surgeon', 'I Know This Much Is True']
  recommended_books_dist = [0.8, 0.77, 0.77, 0.77]
  for i in range(2):
    if recommends[1][i][0] not in recommended_books:
      test_pass = False
    if abs(recommends[1][i][1] - recommended_books_dist[i]) >= 0.05:
      test_pass = False
  if test_pass:
    print("You passed the challenge! 🎉🎉🎉🎉🎉")
  else:
    print("You haven't passed yet. Keep trying!")

test_book_recommendation()